Un decorador es esencialmente una función de alto orden (HOF) que toma como argumento de entrada la función a la que decora y se hace lo siguiente:


In [68]:
def mi_decorador(funcion):
    ...

@mi_decorador
def mi_funcion(args, kwargs):
    ...

# En el tiempo de decoración (bloque inmediatamente superior con la @), se hace:
#     mi_funcion = mi_decorador(mi_funcion)

In [1]:
def pick_roses(color):
    return f'Rosas {color}.'


def get_flowers(picker):  # Ejemplo de funcion de alto orden
    return picker('rojas')


get_flowers(pick_roses)

'Rosas rojas.'

In [21]:
def decorator(funcion):
    print(f'inside decorator body with decorated function {funcion.__name__}')
    
    def wrapped(*args, **kwargs):
        print(f"inside inner wrapper function with args: {args} and kwargs: {kwargs}")
        funcion(*args, **kwargs)  # Al parecer se puede omitir el return
        
    return wrapped

In [22]:
@decorator
def mi_funcion(a,b,c, **kw):
    print(f'Dentro de mi_funcion()')

inside decorator body with decorated function mi_funcion


In [23]:
print("finished decorating my_funcion()")
mi_funcion(1,9, 3, d=5,e=4)
print("immediately after my_function() line")

finished decorating my_funcion()
inside inner wrapper function with args: (1, 9, 3) and kwargs: {'d': 5, 'e': 4}
Dentro de mi_funcion()
immediately after my_function() line


In [2]:
class Decorator(object):
    
    def __init__(self, funcion):
        print(f'inside __init__ with decorated function {funcion.__name__}')
        self.funcion = funcion
        
    def __call__(self, *args, **kwargs):
        print(f"inside __call__() with args: {args} and kwargs: {kwargs}")
        self.funcion(*args, **kwargs)


In [3]:
@Decorator
def mi_funcion(a,b,c, **kw):
    print(f'Dentro de mi_funcion()')

inside __init__ with decorated function mi_funcion


In [4]:
print("finished decorating my_funcion()")
mi_funcion(1,9, 3, d=5,e=4)
print("immediately after my_function() line")

finished decorating my_funcion()
inside __call__() with args: (1, 9, 3) and kwargs: {'d': 5, 'e': 4}
Dentro de mi_funcion()
immediately after my_function() line


In [39]:
def mi_decorador(*decorador_args, **decorador_kwargs):
    print(f'Dentro de mi_decorador con args {decorador_args} y kwargs {decorador_kwargs}')
    
    def inner(funcion):
        print(f'Dentro de inner() con función {funcion.__name__}')
        
        def wrapped(*args, **kwargs):
            print(f'Dentro de wrapped() con args {args} y kwargs {kwargs}')
            return funcion(*args, **kwargs)
        
        return wrapped
    
    return inner

In [40]:
@mi_decorador('decorador_arg_1', 'decorador_arg_2', a=3, b=5)
def fun(*args, **kwargs):
    print(f"inside fun() with args: {args} and kwargs: {kwargs}")

Dentro de mi_decorador con args ('decorador_arg_1', 'decorador_arg_2') y kwargs {'a': 3, 'b': 5}
Dentro de inner() con función fun


In [41]:
print("finished decorating fun()")
fun(1,9, 3, d=5,e=4)
print("immediately after mi_funcion() line")

finished decorating fun()
Dentro de wrapped() con args (1, 9, 3) y kwargs {'d': 5, 'e': 4}
inside fun() with args: (1, 9, 3) and kwargs: {'d': 5, 'e': 4}
immediately after mi_funcion() line


In [47]:
class MiDecorador(object):
    
    def __init__(self, *decorator_args, **decorator_kwargs):
        print(f'Dentro de __init__() with decorator args {decorator_args} and decorator kwargs {decorator_kwargs}')
        
    def __call__(self, funcion):
        print(f"inside __call__() with function {funcion.__name__}")
        
        def wrapped(self, *args, **kwargs):
            print(f'Dentro de wrapped() con args {args} y kwargs {kwargs}')
            return funcion(*args, **kwargs)
        
        return wrapped

In [48]:
@MiDecorador('decorator_arg_1', 'decorator_rg_2', a=3, b=4)
def fun(*args, **kwargs):
    print(f"inside fun() with args: {args} and kwargs: {kwargs}")

Dentro de __init__() with decorator args ('decorator_arg_1', 'decorator_rg_2') and decorator kwargs {'a': 3, 'b': 4}
inside __call__() with function fun


In [44]:
print("finished decorating mi_funcion()")
fun(1,9, 3, d=5,e=4)
print("immediately after mi_funcion() line")

finished decorating mi_funcion()
Dentro de wrapped() con args (9, 3) y kwargs {'d': 5, 'e': 4}
inside fun() with args: (9, 3) and kwargs: {'d': 5, 'e': 4}
immediately after mi_funcion() line


In [59]:
def time_this(function):
    def wrapped(*args, **kwargs):
        print('___Timer starts___')
        from datetime import datetime
        before = datetime.now()
        x = function(*args, **kwargs)
        after = datetime.now()
        print('**Elapsed interval: {}**'.format(after - before))
        return x
    
    return wrapped

In [60]:
def time_all_class_methods(class_):
    # decoration body - doing nothing really since we need to wait until the decorated object is instantiated
    class Wrapper:
        def __init__(self, *args, **kwargs):
            print(f"__init__() called with args: {args} and kwargs: {kwargs}")
            self.decorated_obj = class_(*args, **kwargs)
            
        def __getattribute__(self, s):
            try:
                x = super().__getattribute__(s)
                return x
            except AttributeError:
                pass
            
            x = self.decorated_obj.__getattribute__(s)
            if type(x) == type(self.__init__):
                print(f"attribute belonging to decorated_obj: {s}")
                return time_this(x)  # this is equivalent of just decorating the method with time_this
            else:
                return x
    
    return Wrapper  # decoration ends here

In [64]:
print("before decoration")

@time_all_class_methods
class MyClass:
    def __init__(self):
        import time
        print('entering MyClass.__init__')
        time.sleep(1.8)
        print("exiting MyClass.__init__")
        
    def method_x(self):
        print("entering MyClass.method_x")
        import time
        time.sleep(0.7)
        print("exiting MyClass.method_x")
        
    def method_y(self):
        print("entering MyClass.method_y")
        import time
        time.sleep(1.2)
        print("exiting MyClass.method_y")

print("after decoration")

before decoration
after decoration


In [65]:
instance = MyClass()

__init__() called with args: () and kwargs: {}
entering MyClass.__init__
exiting MyClass.__init__


In [66]:
print("object created")
instance.method_x()
instance.method_y()

object created
attribute belonging to decorated_obj: method_x
___Timer starts___
entering MyClass.method_x
exiting MyClass.method_x
**Elapsed interval: 0:00:00.700873**
attribute belonging to decorated_obj: method_y
___Timer starts___
entering MyClass.method_y
exiting MyClass.method_y
**Elapsed interval: 0:00:01.201363**


In [50]:
# Ejemplo de funcionamiento de args y kwargs
def fun(*args, **kwargs):
    print(f"inside fun() with args: {args} and kwargs: {kwargs}")

In [51]:
fun(3,4,a=7,b=9)

inside fun() with args: (3, 4) and kwargs: {'a': 7, 'b': 9}


In [57]:
c=(3,4)
d={'e':1, 'f':2}

In [58]:
fun(*c,a=7,b=9,**d)

inside fun() with args: (3, 4) and kwargs: {'a': 7, 'b': 9, 'e': 1, 'f': 2}
